In [1]:
import pandas as pd
class BaseHedgeFeature:
	def __init__(self, equities):
		if equities == None:
			self.equity_names = []
		elif type(equities) is str:
			self.equity_names = [equities]
		else:
			self.equity_names = equities

	def set_model_data_path(self, model_data_path):
		self.model_data_path = model_data_path

	def get_hedge_names(self):
		return self.equity_names

	def get_hedge_entry(self, tradelog):
		if len(self.equity_names) == 0:
			return pd.DataFrame([])

		df = pd.read_csv(f"{self.model_data_path}/{self.equity_names[0]}-1d.csv", parse_dates=['Date'])

		hedges = []
		for _, log in tradelog.iterrows():
			row = {col: 0.0 for col in tradelog.columns}
			row['Order no'] = log['Order no']
			row['Equity Name'] = self.equity_names[0]
			row['Trade'] = 'Long' if log['Trade'] == 'Long' else 'Short'
			row['Entry Time'] = log['Entry Time']
			row['Entry Price'] = df[df['Date'] == log['Entry Time']]['Open'].values[0]
			row['Position Size'] = log['Position Size']
			row['Quantity'] = round(row['Position Size'] / row['Entry Price'], 3)
			hedges.append(row)

		return pd.DataFrame(hedges ,columns=tradelog.columns)

	def get_hedge_exit(self, tradelog, hedgelog):
		if len(self.equity_names) == 0:
			return hedgelog

		df = pd.read_csv(f"{self.model_data_path}/{self.equity_names[0]}-1d.csv", parse_dates=['Date'])

		for _, log in tradelog.iterrows():
			hedgelog.loc[hedgelog['Order no'] == log['Order no'], 'Exit Time'] = log['Exit Time']
			hedgelog.loc[hedgelog['Order no'] == log['Order no'], 'Exit Price'] = df[df['Date'] == log['Exit Time']]['Open'].values[0]
			hedgelog.loc[hedgelog['Order no'] == log['Order no'], 'Exit Type'] = log['Exit Type']
			hedgelog.loc[hedgelog['Order no'] == log['Order no'], 'Holding Period'] = log['Exit Time'] - log['Entry Time']

		return hedgelog

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2023-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(10)
        self.hedging_system = BaseHedgeFeature('SPY')

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  37 of 37 completed


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TakeProfitStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2023-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Stop Loss Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TakeProfitStopLoss(200, 10)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  36 of 36 completed
